In [1]:
import pandas as pd
import ast
import numpy as np

In [2]:
data_2018 = pd.read_csv('output_2018.csv',low_memory=False)
data_2019 = pd.read_csv('output_2019.csv',low_memory=False)
data_2020 = pd.read_csv('output_2020.csv',low_memory=False)
data_2021 = pd.read_csv('output_2021.csv',low_memory=False)
data_2022 = pd.read_csv('output_2022.csv',low_memory=False)
data_2023 = pd.read_csv('output_2023.csv',low_memory=False)

for year in [2018, 2019, 2020, 2021, 2022, 2023]:
  print(eval(f'data_{year}').shape)

(2792, 310)
(3082, 310)
(3393, 318)
(3815, 302)
(4244, 288)
(2890, 297)


In [3]:
data = pd.concat([data_2018, data_2019, data_2020, data_2021, data_2022, data_2023], ignore_index=True)
data.shape

(20216, 333)

In [4]:
# Rename columns
new_columns = {
    'title': 'title',
    'abstracts-retrieval-response.subject-areas.subject-area': 'subject_areas',
    'abstracts-retrieval-response.item.bibrecord.head.source.publicationdate.year': 'publication_year',
    'aggregationType': 'aggregation_type',
    'abstracts-retrieval-response.item.bibrecord.tail.bibliography.@refcount': 'reference_count',
    'abstracts-retrieval-response.coredata.openaccess': 'open_access',
    'publisher': 'publisher',
    'funding-list.@has-funding-info': 'has_funding_info',
    'abstracts-retrieval-response.authkeywords.author-keyword' :'keywords',
    'abstracts-retrieval-response.coredata.citedby-count': 'citation_count'
}

# Rename the columns in the DataFrame
data.rename(columns=new_columns, inplace=True)


# Select the target variable
target_column = 'open_access'

# Select the features
features = list(new_columns.values())
features.remove(target_column)


# Check if all columns exist in the dataframe
missing_columns = set(features) - set(data.columns)
if missing_columns:
    print(f"Warning: The following features are not in the dataframe: {missing_columns}")
    features = [col for col in features if col in data.columns]


# Create a new DataFrame with selected features and the target variable
selected_data = data[['filename']+features + [target_column]]

# Display the first few rows of the selected data
selected_data.head()

,filename,title,subject_areas,publication_year,aggregation_type,reference_count,publisher,has_funding_info,keywords,citation_count,open_access
0,201800282,Recent developments in bifunctional air electr...,"[{'@_fa': 'true', '$': 'Renewable Energy, Sust...",2018,Journal,89.0,Elsevier Ltd,1.0,"[{'@_fa': 'true', '$': 'Bifunctional air elect...",29.0,0.0
1,201801350,The benefit of punishment sensitivity on motor...,"[{'@_fa': 'true', '$': 'Social Psychology', '@...",2018,Journal,44.0,Blackwell Publishing Ltd,NaN,"[{'@_fa': 'true', '$': 'anxiety'}, {'@_fa': 't...",2.0,2.0
2,201801162,MAGIC biomarkers predict long-term outcomes fo...,"[{'@_fa': 'true', '$': 'Biochemistry', '@code'...",2018,Journal,30.0,American Society of Hematology,1.0,NaN,117.0,1.0
3,201801506,Are endocrine disrupting compounds environment...,"[{'@_fa': 'true', '$': 'Endocrinology', '@code...",2018,Journal,128.0,Academic Press Inc.apjcs@harcourt.com,1.0,"[{'@_fa': 'true', '$': 'Autism'}, {'@_fa': 'tr...",51.0,1.0
4,201802383,Probabilistic regular grammar inference algori...,"[{'@_fa': 'true', '$': 'Information Systems', ...",2018,Conference Proceeding,9.0,International Workshop on Computer Science and...,1.0,"[{'@_fa': 'true', '$': 'Grammar inference'}, {...",0.0,NaN


In [5]:
missing_values = selected_data.isnull().sum()
print("Missing Values per Column:")
print(missing_values)

Missing Values per Column:
filename               0
title                  1
subject_areas          0
publication_year       0
aggregation_type       0
reference_count      411
publisher             11
has_funding_info    3428
keywords            3804
citation_count         4
open_access         1668
dtype: int64


In [6]:
selected_data = selected_data.dropna(subset=['title'])
selected_data = selected_data.dropna(subset=['open_access'])

In [7]:
# Fill missing 'Reference Count' with the mean
selected_data['reference_count'] = selected_data['reference_count'].fillna(selected_data['reference_count'].mean())

# Verify the imputation
missing_values_after_imputation_refCount = selected_data.isnull().sum()
print("Missing Values per Column:")
print(missing_values_after_imputation_refCount)

Missing Values per Column:
filename               0
title                  0
subject_areas          0
publication_year       0
aggregation_type       0
reference_count        0
publisher              8
has_funding_info    3016
keywords            3545
citation_count         2
open_access            0
dtype: int64


In [8]:
selected_data['citation_count'] = selected_data['citation_count'].fillna(selected_data['citation_count'].mean())

# Verify the imputation
missing_values_after_citation_imputation = selected_data.isnull().sum()
print("Missing Values per Column After Imputation:")
print(missing_values_after_citation_imputation)


Missing Values per Column After Imputation:
filename               0
title                  0
subject_areas          0
publication_year       0
aggregation_type       0
reference_count        0
publisher              8
has_funding_info    3016
keywords            3545
citation_count         0
open_access            0
dtype: int64


In [9]:
selected_data.dropna(subset=['publisher'], inplace=True)

# Verify that rows with missing 'Publisher' have been removed
missing_values_after_publisher_drop = selected_data.isnull().sum()
print("Missing Values per Column After Dropping rows with missing Publisher:")
print(missing_values_after_publisher_drop)

Missing Values per Column After Dropping rows with missing Publisher:
filename               0
title                  0
subject_areas          0
publication_year       0
aggregation_type       0
reference_count        0
publisher              0
has_funding_info    3013
keywords            3544
citation_count         0
open_access            0
dtype: int64


In [10]:
selected_data['has_funding_info'] = selected_data['has_funding_info'].fillna(0)

# Verify the imputation
missing_values_after_imputation_funding = selected_data.isnull().sum()
print("Missing Values per Column After Imputation:")
print(missing_values_after_imputation_funding)

Missing Values per Column After Imputation:
filename               0
title                  0
subject_areas          0
publication_year       0
aggregation_type       0
reference_count        0
publisher              0
has_funding_info       0
keywords            3544
citation_count         0
open_access            0
dtype: int64


In [11]:
selected_data.describe()

,filename,publication_year,reference_count,has_funding_info,citation_count,open_access
count,1.854000e+04,18540.000000,18540.000000,18540.000000,18540.000000,18540.000000
mean,2.020672e+08,2020.654908,47.198624,0.837487,9.930467,0.588889
std,1.625926e+05,1.624594,53.792437,0.368931,28.195993,0.573829
min,2.018000e+08,2018.000000,1.000000,0.000000,0.000000,0.000000
25%,2.019024e+08,2019.000000,26.000000,1.000000,1.000000,0.000000
50%,2.021010e+08,2021.000000,40.000000,1.000000,3.000000,1.000000
75%,2.022022e+08,2022.000000,56.000000,1.000000,10.000000,1.000000
max,2.023029e+08,2023.000000,4084.000000,1.000000,1356.000000,2.000000


In [13]:
def process_subject_areas(subject_areas_str):
    try:
        subject_areas = ast.literal_eval(subject_areas_str)
        if isinstance(subject_areas, list):
            subject_name = []
            subject_code = []
            subject_abbreviation = []
            for subject in subject_areas:
                subject_name.append(subject.get('$', ''))
                subject_code.append(subject.get('@code', ''))
                subject_abbreviation.append(subject.get('@abbrev', ''))
            return ', '.join(subject_name), ', '.join(subject_code), ', '.join(subject_abbreviation)
        else:
            return '', '', ''
    except (SyntaxError, ValueError):
        return '', '', ''

# Apply the function to the 'Subject Areas' column
selected_data[['subject_name', 'subject_code', 'subject_abbreviation']] = selected_data['subject_areas'].apply(lambda x: pd.Series(process_subject_areas(x)))

# Display the updated DataFrame
selected_data.head()

,filename,title,subject_areas,publication_year,aggregation_type,reference_count,publisher,has_funding_info,keywords,citation_count,open_access,subject_name,subject_code,subject_abbreviation
0,201800282,Recent developments in bifunctional air electr...,"[{'@_fa': 'true', '$': 'Renewable Energy, Sust...",2018,Journal,89.0,Elsevier Ltd,1.0,"[{'@_fa': 'true', '$': 'Bifunctional air elect...",29.0,0.0,"Renewable Energy, Sustainability and the Envir...","2105, 2103, 3104, 2102","ENER, ENER, PHYS, ENER"
1,201801350,The benefit of punishment sensitivity on motor...,"[{'@_fa': 'true', '$': 'Social Psychology', '@...",2018,Journal,44.0,Blackwell Publishing Ltd,0.0,"[{'@_fa': 'true', '$': 'anxiety'}, {'@_fa': 't...",2.0,2.0,Social Psychology,3207,PSYC
2,201801162,MAGIC biomarkers predict long-term outcomes fo...,"[{'@_fa': 'true', '$': 'Biochemistry', '@code'...",2018,Journal,30.0,American Society of Hematology,1.0,NaN,117.0,1.0,"Biochemistry, Immunology, Hematology, Cell Bio...","1303, 2403, 2720, 1307","BIOC, IMMU, MEDI, BIOC"
3,201801506,Are endocrine disrupting compounds environment...,"[{'@_fa': 'true', '$': 'Endocrinology', '@code...",2018,Journal,128.0,Academic Press Inc.apjcs@harcourt.com,1.0,"[{'@_fa': 'true', '$': 'Autism'}, {'@_fa': 'tr...",51.0,1.0,"Endocrinology, Endocrine and Autonomic Systems...","1310, 2807, 2802","BIOC, NEUR, NEUR"
5,201801734,Optimization of cinnamon oil nanoemulsions usi...,"[{'@_fa': 'true', '$': 'Electronic, Optical an...",2018,Journal,43.0,Academic Press Inc.apjcs@harcourt.com,1.0,"[{'@_fa': 'true', '$': 'Cinnamon oil'}, {'@_fa...",93.0,0.0,"Electronic, Optical and Magnetic Materials, Bi...","2504, 2502, 2508, 1505","MATE, MATE, MATE, CENG"


In [15]:
# Calculate the age of the work
current_year = 2024  # Replace with the current year
selected_data['age_of_paper'] = current_year - selected_data['publication_year']

# Display the updated DataFrame with the 'Age' column
selected_data.head()

,filename,title,subject_areas,publication_year,aggregation_type,reference_count,publisher,has_funding_info,keywords,citation_count,open_access,subject_name,subject_code,subject_abbreviation,age_of_paper
0,201800282,Recent developments in bifunctional air electr...,"[{'@_fa': 'true', '$': 'Renewable Energy, Sust...",2018,Journal,89.0,Elsevier Ltd,1.0,"[{'@_fa': 'true', '$': 'Bifunctional air elect...",29.0,0.0,"Renewable Energy, Sustainability and the Envir...","2105, 2103, 3104, 2102","ENER, ENER, PHYS, ENER",6
1,201801350,The benefit of punishment sensitivity on motor...,"[{'@_fa': 'true', '$': 'Social Psychology', '@...",2018,Journal,44.0,Blackwell Publishing Ltd,0.0,"[{'@_fa': 'true', '$': 'anxiety'}, {'@_fa': 't...",2.0,2.0,Social Psychology,3207,PSYC,6
2,201801162,MAGIC biomarkers predict long-term outcomes fo...,"[{'@_fa': 'true', '$': 'Biochemistry', '@code'...",2018,Journal,30.0,American Society of Hematology,1.0,NaN,117.0,1.0,"Biochemistry, Immunology, Hematology, Cell Bio...","1303, 2403, 2720, 1307","BIOC, IMMU, MEDI, BIOC",6
3,201801506,Are endocrine disrupting compounds environment...,"[{'@_fa': 'true', '$': 'Endocrinology', '@code...",2018,Journal,128.0,Academic Press Inc.apjcs@harcourt.com,1.0,"[{'@_fa': 'true', '$': 'Autism'}, {'@_fa': 'tr...",51.0,1.0,"Endocrinology, Endocrine and Autonomic Systems...","1310, 2807, 2802","BIOC, NEUR, NEUR",6
5,201801734,Optimization of cinnamon oil nanoemulsions usi...,"[{'@_fa': 'true', '$': 'Electronic, Optical an...",2018,Journal,43.0,Academic Press Inc.apjcs@harcourt.com,1.0,"[{'@_fa': 'true', '$': 'Cinnamon oil'}, {'@_fa...",93.0,0.0,"Electronic, Optical and Magnetic Materials, Bi...","2504, 2502, 2508, 1505","MATE, MATE, MATE, CENG",6


In [16]:
selected_data.head()

,filename,title,subject_areas,publication_year,aggregation_type,reference_count,publisher,has_funding_info,keywords,citation_count,open_access,subject_name,subject_code,subject_abbreviation,age_of_paper
0,201800282,Recent developments in bifunctional air electr...,"[{'@_fa': 'true', '$': 'Renewable Energy, Sust...",2018,Journal,89.0,Elsevier Ltd,1.0,"[{'@_fa': 'true', '$': 'Bifunctional air elect...",29.0,0.0,"Renewable Energy, Sustainability and the Envir...","2105, 2103, 3104, 2102","ENER, ENER, PHYS, ENER",6
1,201801350,The benefit of punishment sensitivity on motor...,"[{'@_fa': 'true', '$': 'Social Psychology', '@...",2018,Journal,44.0,Blackwell Publishing Ltd,0.0,"[{'@_fa': 'true', '$': 'anxiety'}, {'@_fa': 't...",2.0,2.0,Social Psychology,3207,PSYC,6
2,201801162,MAGIC biomarkers predict long-term outcomes fo...,"[{'@_fa': 'true', '$': 'Biochemistry', '@code'...",2018,Journal,30.0,American Society of Hematology,1.0,NaN,117.0,1.0,"Biochemistry, Immunology, Hematology, Cell Bio...","1303, 2403, 2720, 1307","BIOC, IMMU, MEDI, BIOC",6
3,201801506,Are endocrine disrupting compounds environment...,"[{'@_fa': 'true', '$': 'Endocrinology', '@code...",2018,Journal,128.0,Academic Press Inc.apjcs@harcourt.com,1.0,"[{'@_fa': 'true', '$': 'Autism'}, {'@_fa': 'tr...",51.0,1.0,"Endocrinology, Endocrine and Autonomic Systems...","1310, 2807, 2802","BIOC, NEUR, NEUR",6
5,201801734,Optimization of cinnamon oil nanoemulsions usi...,"[{'@_fa': 'true', '$': 'Electronic, Optical an...",2018,Journal,43.0,Academic Press Inc.apjcs@harcourt.com,1.0,"[{'@_fa': 'true', '$': 'Cinnamon oil'}, {'@_fa...",93.0,0.0,"Electronic, Optical and Magnetic Materials, Bi...","2504, 2502, 2508, 1505","MATE, MATE, MATE, CENG",6


In [17]:
# Drop the 'subject_areas' column
selected_data = selected_data.drop('subject_areas', axis=1)

In [18]:
selected_data['subject_code'].head()

0    2105, 2103, 3104, 2102
1                      3207
2    1303, 2403, 2720, 1307
3          1310, 2807, 2802
5    2504, 2502, 2508, 1505
Name: subject_code, dtype: object

In [19]:
subject_to_supergroup = {
    '1000': 'Multidisciplinary',
    '1100': 'Life Sciences',  # Agricultural and Biological Sciences
    '1200': 'Social Sciences',  # Arts and Humanities
    '1300': 'Life Sciences',  # Biochemistry, Genetics and Molecular Biology
    '1400': 'Social Sciences',  # Business, Management, and Accounting
    '1500': 'Physical Sciences',  # Chemical Engineering
    '1600': 'Physical Sciences',  # Chemistry
    '1700': 'Physical Sciences',  # Computer Science
    '1800': 'Social Sciences',  # Decision Sciences
    '1900': 'Physical Sciences',  # Earth and Planetary Sciences
    '2000': 'Social Sciences',  # Economics, Econometrics and Finance
    '2100': 'Physical Sciences',  # Energy
    '2200': 'Physical Sciences',  # Engineering
    '2300': 'Physical Sciences',  # Environmental Science
    '2400': 'Life Sciences',  # Immunology and Microbiology
    '2500': 'Physical Sciences',  # Materials Science
    '2600': 'Physical Sciences',  # Mathematics
    '2700': 'Health Sciences',  # Medicine
    '2800': 'Life Sciences',  # Neuroscience
    '2900': 'Health Sciences',  # Nursing
    '3000': 'Life Sciences',  # Pharmacology, Toxicology, and Pharmaceutics
    '3100': 'Physical Sciences',  # Physics and Astronomy
    '3200': 'Social Sciences',  # Psychology
    '3300': 'Social Sciences',  # Social Sciences
    '3400': 'Health Sciences',  # Veterinary
    '3500': 'Health Sciences',  # Dentistry
    '3600': 'Health Sciences'  # Health Professions
}

In [20]:
def map_to_supergroup(subject_codes):
    codes = subject_codes.split(', ')  # Split subject codes into a list
    supergroups = []
    
    for code in codes:
        prefix = code[:2]  # Get the first two digits of the code
        matching_supergroup = None
        
        # Check if the prefix matches any key in the subject_to_supergroup dictionary
        for key, value in subject_to_supergroup.items():
            if key.startswith(prefix):  # If the first two digits match
                matching_supergroup = value
                break
        
        supergroups.append(matching_supergroup if matching_supergroup else 'Unknown')  # Append result
    
    return ', '.join(set(supergroups))  # Join the different supergroups into a result

# Use the function to map subject_code to Supergroup
selected_data['supergroup'] = selected_data['subject_code'].apply(map_to_supergroup)


In [21]:
selected_data['supergroup'].value_counts()

supergroup
Physical Sciences                                                     6443
Health Sciences                                                       3889
Life Sciences                                                         1900
Health Sciences, Life Sciences                                        1787
Physical Sciences, Life Sciences                                      1129
Social Sciences                                                        963
Multidisciplinary                                                      924
Physical Sciences, Social Sciences                                     782
Physical Sciences, Health Sciences                                     208
Health Sciences, Social Sciences                                       152
Physical Sciences, Life Sciences, Health Sciences                       92
Physical Sciences, Life Sciences, Social Sciences                       80
Health Sciences, Life Sciences, Social Sciences                         52
Life Sciences,

In [22]:
selected_data['supergroup'] = selected_data['supergroup'].apply(lambda x: 'Multidisciplinary' if ',' in x else x)
selected_data['supergroup'].value_counts()

supergroup
Physical Sciences    6443
Multidisciplinary    5345
Health Sciences      3889
Life Sciences        1900
Social Sciences       963
Name: count, dtype: int64

In [23]:
def extract_keywords(row):
    if pd.isna(row):  # ถ้าข้อมูลเป็น NaN ให้คืนค่า "No keyword"
        return "No keyword"
    try:
        # แปลง string เป็น list ของ dictionary
        items = ast.literal_eval(row)
        keywords = [item['$'] for item in items if '$' in item]
        return ", ".join(keywords) if keywords else "No keyword"
    except (ValueError, SyntaxError, TypeError):  # จัดการข้อผิดพลาด
        return "No keyword"


selected_data['keywords'] = selected_data['keywords'].apply(extract_keywords)



In [24]:
selected_data['keywords'].head()

0    Bifunctional air electrode, Catalyst support, ...
1    anxiety, defensive distance, performance, rein...
2                                           No keyword
3    Autism, Endocrine disrupting compounds, Epigen...
5    Cinnamon oil, Nanoemulsions, Natural product, ...
Name: keywords, dtype: object

In [25]:
selected_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18540 entries, 0 to 20215
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   filename              18540 non-null  int64  
 1   title                 18540 non-null  object 
 2   publication_year      18540 non-null  int64  
 3   aggregation_type      18540 non-null  object 
 4   reference_count       18540 non-null  float64
 5   publisher             18540 non-null  object 
 6   has_funding_info      18540 non-null  float64
 7   keywords              18540 non-null  object 
 8   citation_count        18540 non-null  float64
 9   open_access           18540 non-null  float64
 10  subject_name          18540 non-null  object 
 11  subject_code          18540 non-null  object 
 12  subject_abbreviation  18540 non-null  object 
 13  age_of_paper          18540 non-null  int64  
 14  supergroup            18540 non-null  object 
dtypes: float64(4), int64(3),

In [26]:
# Reorder the columns
new_column_order = ['filename','title','subject_name','subject_abbreviation','subject_code','supergroup','keywords','publication_year', 'age_of_paper','aggregation_type', 'reference_count', 'publisher', 'has_funding_info', 'citation_count','open_access']
selected_data = selected_data[new_column_order]

# Display the updated DataFrame
selected_data.head()

,filename,title,subject_name,subject_abbreviation,subject_code,supergroup,keywords,publication_year,age_of_paper,aggregation_type,reference_count,publisher,has_funding_info,citation_count,open_access
0,201800282,Recent developments in bifunctional air electr...,"Renewable Energy, Sustainability and the Envir...","ENER, ENER, PHYS, ENER","2105, 2103, 3104, 2102",Physical Sciences,"Bifunctional air electrode, Catalyst support, ...",2018,6,Journal,89.0,Elsevier Ltd,1.0,29.0,0.0
1,201801350,The benefit of punishment sensitivity on motor...,Social Psychology,PSYC,3207,Social Sciences,"anxiety, defensive distance, performance, rein...",2018,6,Journal,44.0,Blackwell Publishing Ltd,0.0,2.0,2.0
2,201801162,MAGIC biomarkers predict long-term outcomes fo...,"Biochemistry, Immunology, Hematology, Cell Bio...","BIOC, IMMU, MEDI, BIOC","1303, 2403, 2720, 1307",Multidisciplinary,No keyword,2018,6,Journal,30.0,American Society of Hematology,1.0,117.0,1.0
3,201801506,Are endocrine disrupting compounds environment...,"Endocrinology, Endocrine and Autonomic Systems...","BIOC, NEUR, NEUR","1310, 2807, 2802",Life Sciences,"Autism, Endocrine disrupting compounds, Epigen...",2018,6,Journal,128.0,Academic Press Inc.apjcs@harcourt.com,1.0,51.0,1.0
5,201801734,Optimization of cinnamon oil nanoemulsions usi...,"Electronic, Optical and Magnetic Materials, Bi...","MATE, MATE, MATE, CENG","2504, 2502, 2508, 1505",Physical Sciences,"Cinnamon oil, Nanoemulsions, Natural product, ...",2018,6,Journal,43.0,Academic Press Inc.apjcs@harcourt.com,1.0,93.0,0.0


In [27]:
# Convert 'Reference Count', 'Publication Year', 'Has Funding Info', and 'Citation Count' to integers
selected_data['reference_count'] = selected_data['reference_count'].astype(int)
selected_data['open_access'] = selected_data['open_access'].astype(int)
selected_data['publication_year'] = selected_data['publication_year'].astype(int)
selected_data['has_funding_info'] = selected_data['has_funding_info'].astype(int)
selected_data['citation_count'] = selected_data['citation_count'].astype(int)

selected_data.dtypes

filename                 int64
title                   object
subject_name            object
subject_abbreviation    object
subject_code            object
supergroup              object
keywords                object
publication_year         int64
age_of_paper             int64
aggregation_type        object
reference_count          int64
publisher               object
has_funding_info         int64
citation_count           int64
open_access              int64
dtype: object

In [28]:
data_web_to_science = pd.read_csv('web_to_science.csv',low_memory=False)

In [29]:
data_web_to_science.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1915 entries, 0 to 1914
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1915 non-null   int64 
 1   title             1915 non-null   object
 2   publication_year  1915 non-null   int64 
 3   age_of_paper      1915 non-null   int64 
 4   aggregation_type  1915 non-null   object
 5   reference_count   1915 non-null   int64 
 6   open_access       1915 non-null   int64 
 7   has_funding_info  488 non-null    object
 8   citation_count    1915 non-null   int64 
dtypes: int64(6), object(3)
memory usage: 134.8+ KB


In [30]:
data_web_to_science = data_web_to_science[data_web_to_science['publication_year'] >= 2018]
data_web_to_science.reset_index(drop=True, inplace=True)

In [31]:
print(data_web_to_science.describe())

        Unnamed: 0  publication_year  age_of_paper  reference_count  \
count  1232.000000       1232.000000   1232.000000      1232.000000   
mean   1126.357143       2022.845779      1.154221        48.827110   
std     610.593997          1.547698      1.547698        50.080538   
min       0.000000       2018.000000      0.000000         0.000000   
25%     382.500000       2023.000000      0.000000        21.000000   
50%    1298.500000       2023.000000      1.000000        38.000000   
75%    1606.250000       2024.000000      1.000000        62.000000   
max    1914.000000       2024.000000      6.000000       514.000000   

       open_access  citation_count  
count  1232.000000     1232.000000  
mean      1.693994        3.553571  
std       0.461020       11.138609  
min       1.000000        0.000000  
25%       1.000000        0.000000  
50%       2.000000        0.000000  
75%       2.000000        3.000000  
max       2.000000      195.000000  


In [32]:
data_web_to_science.drop(columns=['aggregation_type'], inplace=True)
data_web_to_science.drop(columns=['Unnamed: 0'], inplace=True)

In [33]:
data_web_to_science.head()

,title,publication_year,age_of_paper,reference_count,open_access,has_funding_info,citation_count
0,An hybrid soft attention based XGBoost model f...,2023,1,38,2,NaN,1
1,Parasitic egg recognition using convolution an...,2023,1,62,1,Funding\nThis research project was funded by M...,5
2,Spatiotemporal changes of gross primary produc...,2024,0,83,2,NaN,4
3,Cases of Castigation,2024,0,18,2,NaN,0
4,Understanding Underdevelopment: A Study on Sel...,2024,0,34,2,NaN,0


In [35]:
# เลือกคอลัมน์ที่ต้องการจาก selected_data และ data_web_to_science
selected_columns = ['title', 'age_of_paper', 'reference_count', 'citation_count', 'open_access']

# รวมข้อมูล
selected_data_training = pd.concat([selected_data[selected_columns], data_web_to_science[selected_columns]], ignore_index=True)

# แสดงข้อมูลที่รวมแล้ว
selected_data_training.head()

,title,age_of_paper,reference_count,citation_count,open_access
0,Recent developments in bifunctional air electr...,6,89,29,0
1,The benefit of punishment sensitivity on motor...,6,44,2,2
2,MAGIC biomarkers predict long-term outcomes fo...,6,30,117,1
3,Are endocrine disrupting compounds environment...,6,128,51,1
4,Optimization of cinnamon oil nanoemulsions usi...,6,43,93,0


In [36]:
selected_data_training['reference_count_log'] = np.log1p(selected_data_training['reference_count'])  # ใช้ log(1+x)
selected_data_training['citation_count_log'] = np.log1p(selected_data_training['citation_count'])

In [39]:
selected_data_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19772 entries, 0 to 19771
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                19772 non-null  object 
 1   age_of_paper         19772 non-null  int64  
 2   reference_count      19772 non-null  int64  
 3   citation_count       19772 non-null  int64  
 4   open_access          19772 non-null  int64  
 5   reference_count_log  19772 non-null  float64
 6   citation_count_log   19772 non-null  float64
dtypes: float64(2), int64(4), object(1)
memory usage: 1.1+ MB


In [40]:
selected_data_training.to_csv('selected_data_training.csv', index=False)

In [46]:
supergroup_counts = selected_data['supergroup'].value_counts().reset_index()
supergroup_counts.columns = ['supergroup', 'count']
supergroup_counts 
supergroup_counts.to_csv('supergroup_counts.csv', index=False)

In [79]:
import pandas as pd
import ast
from collections import Counter

# แยก keywords ออกเป็น list
selected_data['keywords_list'] = selected_data['keywords'].apply(lambda x: x.split(", ") if x != "No keyword" else [])

# ฟังก์ชั่นเพื่อหา top 10 keywords สำหรับแต่ละปี (แก้ปัญหาตัวพิมพ์ใหญ่-พิมพ์เล็ก)
def top_keywords_by_year(data):
    result = {}
    for year in data['publication_year'].unique():
        # กรองข้อมูลตามปี
        year_data = data[data['publication_year'] == year]
        
        # รวม keywords ทั้งหมดในปีนั้น และแปลงเป็นตัวพิมพ์เล็ก
        all_keywords = [keyword.lower() for keywords in year_data['keywords_list'] for keyword in keywords]
        
        # นับจำนวนแต่ละคำ
        keyword_counts = Counter(all_keywords)
        
        # หา top 10 keyword ในปีนั้น
        result[year] = keyword_counts.most_common(10)
    
    return result

# เรียกใช้ฟังก์ชั่นเพื่อหา top 10 keyword ตามปี
top_10_keywords_by_year = top_keywords_by_year(selected_data)

# เขียนผลลัพธ์ลง CSV แยกไฟล์ตามปี
for year, top_keywords in top_10_keywords_by_year.items():
    # แปลงผลลัพธ์เป็น DataFrame
    df = pd.DataFrame(top_keywords, columns=['Keyword', 'Count'])
    
    # เขียนลงไฟล์ CSV
    filename = f"top_10_keywords_{year}.csv"
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"บันทึก {filename} สำเร็จ!")

# แสดงผลลัพธ์บนหน้าจอ (อาจเป็นตัวเลือกเสริม)
for year, top_keywords in top_10_keywords_by_year.items():
    print(f"Top 10 keywords in {year}:")
    for keyword, count in top_keywords:
        print(f"- {keyword}: {count}")
    print()


บันทึก top_10_keywords_2018.csv สำเร็จ!
บันทึก top_10_keywords_2019.csv สำเร็จ!
บันทึก top_10_keywords_2020.csv สำเร็จ!
บันทึก top_10_keywords_2021.csv สำเร็จ!
บันทึก top_10_keywords_2022.csv สำเร็จ!
บันทึก top_10_keywords_2023.csv สำเร็จ!
Top 10 keywords in 2018:
- thailand: 85
- hadron-hadron scattering (experiments): 58
- inflammation: 29
- cms: 27
- physics: 25
- beyond standard model: 22
- depression: 21
- hiv: 19
- cytotoxicity: 17
- meta-analysis: 17

Top 10 keywords in 2019:
- thailand: 77
- hadron-hadron scattering (experiments): 35
- inflammation: 31
- cms: 22
- hiv: 20
- adsorption: 18
- oxidative stress: 18
- physics: 15
- beyond standard model: 15
- machine learning: 15

Top 10 keywords in 2020:
- thailand: 96
- covid-19: 31
- hadron-hadron scattering (experiments): 31
- inflammation: 28
- asia: 23
- depression: 20
- hiv: 20
- cms: 20
- oxidative stress: 20
- deep learning: 18

Top 10 keywords in 2021:
- thailand: 127
- covid-19: 75
- inflammation: 47
- hadron-hadron scatt

In [105]:

# ฟังก์ชั่นเพื่อดึงและจัดเรียงทุก keyword สำหรับแต่ละปี
def all_keywords_by_year_sorted(data):
    result = {}
    for year in data['publication_year'].unique():
        # กรองข้อมูลตามปี
        year_data = data[data['publication_year'] == year]
        
        # รวม keywords ทั้งหมดในปีนั้น และแปลงเป็นตัวพิมพ์เล็ก
        all_keywords = [keyword.lower() for keywords in year_data['keywords_list'] for keyword in keywords]
        
        # นับจำนวนแต่ละคำและเรียงลำดับจากมากไปน้อย
        keyword_counts = Counter(all_keywords).most_common()
        
        # เก็บผลลัพธ์ที่เรียงแล้ว
        result[year] = keyword_counts
    
    return result

# เรียกใช้ฟังก์ชั่นเพื่อดึงทุก keyword ตามปี
all_keywords_by_year_result = all_keywords_by_year_sorted(selected_data)

# เขียนผลลัพธ์ลง CSV แยกไฟล์ตามปี
for year, keywords in all_keywords_by_year_result.items():
    # แปลงผลลัพธ์เป็น DataFrame
    df = pd.DataFrame(keywords, columns=['Keyword', 'Count'])
    
    # เขียนลงไฟล์ CSV
    filename = f"all_keywords_{year}.csv"
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"บันทึก {filename} สำเร็จ!")

# แสดงผลลัพธ์บนหน้าจอ (อาจเป็นตัวเลือกเสริม)
for year, keywords in all_keywords_by_year_result.items():
    print(f"All keywords in {year}:")
    for keyword, count in keywords:
        print(f"- {keyword}: {count}")
    print()


บันทึก all_keywords_2018.csv สำเร็จ!
บันทึก all_keywords_2019.csv สำเร็จ!
บันทึก all_keywords_2020.csv สำเร็จ!
บันทึก all_keywords_2021.csv สำเร็จ!
บันทึก all_keywords_2022.csv สำเร็จ!
บันทึก all_keywords_2023.csv สำเร็จ!
All keywords in 2018:
- thailand: 85
- hadron-hadron scattering (experiments): 58
- inflammation: 29
- cms: 27
- physics: 25
- beyond standard model: 22
- depression: 21
- hiv: 19
- cytotoxicity: 17
- meta-analysis: 17
- cytokines: 15
- higgs physics: 14
- oxidative stress: 14
- supersymmetry: 13
- chitosan: 13
- immune: 13
- children: 13
- adsorption: 11
- apoptosis: 11
- asia: 11
- penaeus monodon: 11
- mechanical properties: 10
- quality of life: 10
- atrial fibrillation: 10
- deep learning: 9
- poly(lactic acid): 9
- top physics: 9
- graphene: 9
- galaxies: evolution: 8
- psychiatry: 8
- treatment: 8
- schizophrenia: 8
- bipolar disorder: 8
- nanoparticles: 8
- biomass: 8
- prevalence: 8
- taxonomy: 8
- anxiety: 7
- antioxidant: 7
- photocatalysis: 7
- molecular d

In [73]:
selected_data.head()

,filename,title,subject_name,subject_abbreviation,subject_code,supergroup,keywords,publication_year,age_of_paper,aggregation_type,reference_count,publisher,has_funding_info,citation_count,open_access,keywords_list
0,201800282,Recent developments in bifunctional air electr...,"Renewable Energy, Sustainability and the Envir...","ENER, ENER, PHYS, ENER","2105, 2103, 3104, 2102",Physical Sciences,"Bifunctional air electrode, Catalyst support, ...",2018,6,Journal,89,Elsevier Ltd,1,29,0,"[Bifunctional air electrode, Catalyst support,..."
1,201801350,The benefit of punishment sensitivity on motor...,Social Psychology,PSYC,3207,Social Sciences,"anxiety, defensive distance, performance, rein...",2018,6,Journal,44,Blackwell Publishing Ltd,0,2,2,"[anxiety, defensive distance, performance, rei..."
2,201801162,MAGIC biomarkers predict long-term outcomes fo...,"Biochemistry, Immunology, Hematology, Cell Bio...","BIOC, IMMU, MEDI, BIOC","1303, 2403, 2720, 1307",Multidisciplinary,No keyword,2018,6,Journal,30,American Society of Hematology,1,117,1,[]
3,201801506,Are endocrine disrupting compounds environment...,"Endocrinology, Endocrine and Autonomic Systems...","BIOC, NEUR, NEUR","1310, 2807, 2802",Life Sciences,"Autism, Endocrine disrupting compounds, Epigen...",2018,6,Journal,128,Academic Press Inc.apjcs@harcourt.com,1,51,1,"[Autism, Endocrine disrupting compounds, Epige..."
5,201801734,Optimization of cinnamon oil nanoemulsions usi...,"Electronic, Optical and Magnetic Materials, Bi...","MATE, MATE, MATE, CENG","2504, 2502, 2508, 1505",Physical Sciences,"Cinnamon oil, Nanoemulsions, Natural product, ...",2018,6,Journal,43,Academic Press Inc.apjcs@harcourt.com,1,93,0,"[Cinnamon oil, Nanoemulsions, Natural product,..."


In [74]:
selected_data.to_csv('selected_data.csv', index=False)

In [75]:
missing_values = selected_data.isnull().sum()
print("Missing Values per Column:")
print(missing_values)

Missing Values per Column:
filename                0
title                   0
subject_name            0
subject_abbreviation    0
subject_code            0
supergroup              0
keywords                0
publication_year        0
age_of_paper            0
aggregation_type        0
reference_count         0
publisher               0
has_funding_info        0
citation_count          0
open_access             0
keywords_list           0
dtype: int64
